## Valores atípicos

Un valor atípico o outlier es un punto de observación dentro de un conjunto de datos que es diferentes del resto de los puntos.

Estos valores pueden deberse a un error cometido durante el proceso de obtención y recolección de datos o pueden representar una varianza real en los valores del dataset. Dependiendo del caso pueden ser tratados de diferente manera.



## Encontrar valores atípicos


In [ ]:
import pandas as pd
import requests
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Leer csv
df = pd.read_csv('data/wine/winequality-red.csv')

df

Mostrar características estadísticas de los elementos

In [ ]:
df.describe()

### IQR Score

El rango intercuartílico o Interquartile Range es la diferencia entre el tercer y el primer cuartil de una distribución. 

Muestra la tendencia central de los datos.

IQR = Q3 − Q1

Se asume que cualquier valor fuera de este rango es un valor atípico

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
# Rangos normales de datos
rango_inferior = Q1 - 1.5 * IQR
rango_superior = Q3 + 1.5 * IQR

print("Rango superior")
print(rango_inferior)
print("Rango inferior")
print(rango_superior)

In [ ]:
#Mostrar si un valor es atípico o no
df_outliers = (df < rango_inferior) | (df > rango_superior)
df_outliers

In [ ]:
#Mostar mapa de calor para identificar los valores atípicos
sns.heatmap(df_outliers, cbar=False)

### Box plot

Un diagrama de caja o box plot es un gráfico que representa una serie de daots numéricos a partir de sus cuartiles.
<img src="https://upload.wikimedia.org/wikipedia/commons/4/4a/Diagrama_de_caja.jpg">

In [ ]:
# Mostrar boxplot de un atributo
df.boxplot(column='pH')

In [ ]:
# Mostrar boxplot de un atributo dividio por categoría
df.boxplot(column='pH', by='quality')

In [ ]:
# Mostrar boxplot de un atributo
sns.boxplot(x=df['pH'])

In [ ]:
# Mostrar boxplot de un atributo dividio por categoría
sns.boxplot(y=df['pH'],x=df['quality'])

### Scatter Plot

Los gráficos de disperción pueden mostrar visualmente cuando existen datos que se separan de los valores comunes del atributo

In [ ]:
plt.scatter(df.index,df["pH"])
plt.ylabel('pH')
plt.show()

In [ ]:
# Valor del atributo separado por categoría
plt.scatter(df["quality"], df["pH"])
plt.ylabel('pH')
plt.show() 

### Histogram

El histograma muestra la frecuencia en la que aparecen los valores para un atributo específico. Nos permite ver ocurrencias de valores extremos.

In [ ]:
df.pH.hist()

## Tratamiento

### Flooring and Capping basado en cuartiles

Se reemplazan los valores más bajos con el valor correspondiente a uno de los percentiles más bajos y los mal altos por el valor correspondiente a uno de los percentiles más altos.

In [ ]:
pH_q_sup = df['pH'].quantile(0.90)
pH_q_inf = df['pH'].quantile(0.10)
print(pH_q_sup)
print(pH_q_inf)

In [ ]:
df_fc = df.copy(deep = True)
df_fc.loc[df_fc["pH"] <= pH_q_inf,"pH"] = pH_q_inf
df_fc.loc[df_fc["pH"] >= pH_q_sup, "pH"] = pH_q_sup

In [ ]:
plt.scatter(df["quality"], df["pH"])
plt.title('Original')
plt.ylim(min(df['pH']),max(df['pH']))
plt.ylabel('pH')
plt.show() 

plt.scatter(df_fc["quality"], df_fc["pH"])
plt.title('Valores modificados')
plt.ylim(min(df['pH']),max(df['pH']))
plt.ylabel('pH')
plt.show() 

In [ ]:
a = sns.boxplot(y=df['pH'],x=df['quality'])
a.set(ylim=(min(df['pH']),max(df['pH'])))


In [ ]:
a = sns.boxplot(y=df_fc['pH'],x=df_fc['quality'])
a.set(ylim=(min(df['pH']),max(df['pH'])))

### Trimming

Eliminar los elementos que se encuentren fuera de ciertos cuartiles

In [ ]:
pH_q_sup = df['pH'].quantile(0.98)
pH_q_inf = df['pH'].quantile(0.02)
df_tr = df.copy(deep = True)
index = df_tr[(df['pH'] >= pH_q_sup)|(df_tr['pH'] <= pH_q_inf)].index
df_tr.drop(index, inplace=True)

In [ ]:
plt.scatter(df["quality"], df["pH"])
plt.ylim(min(df['pH']),max(df['pH']))
plt.title('Original')
plt.ylabel('pH')
plt.show() 

plt.scatter(df_tr["quality"], df_tr["pH"])
plt.ylim(min(df['pH']),max(df['pH']))
plt.title('Valores modificados')
plt.ylabel('pH')
plt.show() 


In [ ]:
a = sns.boxplot(y=df['pH'],x=df['quality'])
a.set(ylim=(min(df['pH']),max(df['pH'])))

In [ ]:
a = sns.boxplot(y=df_tr['pH'],x=df_tr['quality'])
a.set(ylim=(min(df['pH']),max(df['pH'])))

### IQR Score

Eliminar aquellos elementos que se consideren outliers de acuerdo con el valor IQR

In [ ]:
pH_q_sup = df['pH'].quantile(0.98)
pH_q_inf = df['pH'].quantile(0.02)
df_IQR = df.copy(deep = True)
index = df_IQR[(df['pH'] >= rango_superior['pH'])|(df_IQR['pH'] <= rango_inferior['pH'])].index
df_IQR.drop(index, inplace=True)

In [ ]:
plt.scatter(df["quality"], df["pH"])
plt.ylim(min(df['pH']),max(df['pH']))
plt.title('Original')
plt.ylabel('pH')
plt.show()    

plt.scatter(df_IQR["quality"], df_IQR["pH"])
plt.ylim(min(df['pH']),max(df['pH']))
plt.title('Valores modificados')
plt.ylabel('pH')
plt.show() 



In [ ]:
a = sns.boxplot(y=df['pH'],x=df['quality'])
a.set(ylim=(min(df['pH']),max(df['pH'])))

In [ ]:
a = sns.boxplot(y=df_IQR['pH'],x=df_IQR['quality'])
a.set(ylim=(min(df['pH']),max(df['pH'])))

### Transformación logarítmica

Aplicar una transformación logarítmica sobre los elementos. Esto lo convierte en una distribucion normal pero cambia el valor y por lo tanto el significado de los datos. Es útil para los algoritmos de machine learning que requieren este tipo de distribución.

In [ ]:
df_log = df.copy(deep = True)
df_log["pH"] = np.log(df_log["pH"]) 

In [ ]:
plt.scatter(df["quality"], df["pH"])
plt.title('Original')
plt.ylim(min(df_log['pH']),max(df['pH']))
plt.ylabel('pH')
plt.show() 

plt.scatter(df_log["quality"], df_log["pH"])
plt.title('Valores modificados')
plt.ylim(min(df_log['pH']),max(df['pH']))
plt.ylabel('pH')
plt.show() 

In [ ]:
a = sns.boxplot(y=df['pH'],x=df['quality'])
a.set(ylim=(min(df['pH']),max(df['pH'])))

In [ ]:
a = sns.boxplot(y=df_log['pH'],x=df_log['quality'])
a.set(ylim=(min(df_log['pH']),max(df['pH'])))